In [1]:
from sqlalchemy import create_engine

import dash
# import dash_daq as daq
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.graph_objects as go

import pandas as pd

In [2]:
import os
import sys

dir_actual = os.getcwd()
local = os.path.dirname(os.path.abspath(dir_actual))
sys.path.insert(0, local)
print(local)

/media/Data/suplent/work_lab/fiocruz/AlertaDengue


#### dbdata

In [3]:
engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        "dengueadmin",
        "aldengue",
        "172.25.0.2",
        '25432',
        "dengue",
    )
)

In [4]:
def get_disease_suffix(disease: str):
    """

    :param disease:
    :return:
    """
    return (
        ''
        if disease == 'dengue'
        else '_chik'
        if disease == 'chikungunya'
        else '_zika'
        if disease == 'zika'
        else ''
    )


def get_uf_by_disease(uf, disease):
    """
    Returna contagem de cidades participantes por estado

    :param uf: uf a ser consultada
    :param disease: dengue|chikungunya|zika
    :return: dataframe
    """

    table_name = 'Historico_alerta' + get_disease_suffix(disease)
    weeks = 52

    sql = '''
        SELECT DISTINCT alerta."SE",
            alerta."Rt", alerta."data_iniSE",
            alerta.casos_est, alerta.casos, alerta.nivel, alerta.receptivo,
            alerta.municipio_geocodigo,
            city.nome, city.uf
        FROM "Municipio"."%s" AS alerta
        INNER JOIN "Dengue_global"."Municipio" AS city ON (
                    alerta.municipio_geocodigo=city.geocodigo)        
    WHERE uf='%s'
    AND alerta."data_iniSE" >= NOW() - interval '%s weeks';
    ''' % (
        table_name,
        uf,
        weeks+2
    )

    with engine.connect() as conn:
        return pd.read_sql(sql, conn)

#### Update disease by UF

In [ ]:
df.groupby(["state", "gender"])["last_name"].count()

In [ ]:
def update_figure(selected_uf):
    disease = 'dengue' # < pass this in the dropdown
    df = get_uf_by_disease(selected_uf, disease)
    keys=['casos_est', 'casos']
    df_by_uf = df.groupby(['SE'])[keys].sum()

    print(df_by_uf)
#     
    traces = []


update_figure('Ceará')

##### Create Plotly-Dash-Charts

In [ ]:
app = dash.Dash()

# States
ufs = ['Ceará', 'Rio de Janeiro', 'Paraná', 'Espírito Santo',
       'São Paulo', 'Minas Gerais', 'Rio Grande do Sul', 'Maranhão']

year_options = []
for uf in ufs:
    year_options.append({'label': str(uf), 'value': uf})


app.layout = html.Div([

    # first row
    html.Div(children=[

        # first column of first row
        html.Div(children=[

            dcc.Dropdown(id='state-picker', options=year_options,
                        placeholder="Select a State name"), #multi=True

        ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),

    ], className='row'),
    html.Hr(),
    # second row
    html.Div(children=[

        # first column of second row
        html.Div(children=[

            html.Div(dcc.Graph(id='graph')),

        ], style={'width':'25%', 'display':'inline-block'}),
        # style={'display': 'inline-block', 'vertical-align': 'top', 'margin-right': '3vw', 'margin-top': '3vw'}),

        # second column of second row
        html.Div(children=[

            html.Div(dcc.Graph(id='indicator')),

        ], style={'width':'15%', 'display':'inline-block', 'vertical-align': 'top', 'margin-right': '3vw', 'margin-top': '7vw'}),
        

    ], className='row'),
    html.Hr(),


])

# Traces graph
@app.callback(Output('graph', 'figure'),
              [Input('state-picker', 'value')])
def update_figure(selected_uf):
    disease = 'dengue' # < pass this in the dropdown
    df = get_uf_by_disease(selected_uf, disease)
    keys=['casos_est', 'casos']
    df_by_uf = df.groupby(['SE'])[keys].sum()

    traces = []

    traces.append(
        go.Bar(
            x=df_by_uf.index.map(
                lambda v: '%s/%s' % (str(v)[:4], str(v)[-2:])),
            y=df_by_uf['casos_est'],
            name="Estimated Cases",
            text=df_by_uf.index.map(lambda v: '{}'.format(str(v)[-2:])),
            hoverinfo='text',
            hovertemplate="Semana %{text}<br>Incidência=%{y:1f}",

        ))
    traces.append(
        go.Scatter(
            x=df_by_uf.index.map(
                lambda v: '%s/%s' % (str(v)[:4], str(v)[-2:])),
            y=df_by_uf['casos'],
            name="Cases",
            text=df_by_uf.index.map(lambda v: '{}'.format(str(v)[-2:])),
            hoverinfo='text',
            hovertemplate="Semana %{text}<br>Incidência=%{y:1f}",

        ))

    return {
        'data': traces,
        'layout': go.Layout(
            title='{}'.format(selected_uf),
            
            xaxis={'title': 'Week'},
            yaxis={'title': 'Cases'},
            showlegend = False,
            hovermode='closest'
        )
    }


# Indicator Graph
@app.callback(
    Output('indicator', 'figure'),
    [Input('state-picker', 'value')])
def update_tank(selected_uf):
    disease = 'dengue' # < pass this in the dropdown
    df = get_uf_by_disease(selected_uf, disease)
    traces = []
    
    # Select by uf
    df_uf = df['uf'] == selected_uf
    uf_selected = df[df_uf]
    
    # Count the total cities in uf
    cities_uf = len(uf_selected.nome.unique())
    print(cities_uf)
    
    # Filter nivel
    nivel_uf = 4
    rec_uf = uf_selected['nivel'] == nivel_uf
    rec_selected = uf_selected[rec_uf]
    
    # Filter last week
    last_se = rec_selected.SE.max()
    ant_se = last_se - 1
    df_last_se = rec_selected[rec_selected['SE'] == last_se].nivel.count()    
    df_ant_se = rec_selected[rec_selected['SE'] == ant_se].nivel.count()

    traces.append(
        go.Indicator(
            domain = {'x': [0, 1], 'y': [0, 1]},
            value = df_last_se,
            mode = "gauge+number+delta",
            title = {'text': "Receptivity"},
            delta = {'reference': df_ant_se},
            gauge = {'axis': {'range': [None, cities_uf]},
                     'steps' : [
                         {'range': [0, 250], 'color': "lightgray"},
                         {'range': [250, 400], 'color': "gray"}],
                     'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': cities_uf}}))


#     return df_last_se.nivel.count()
    return {
        'data': traces,
    }


if __name__ == '__main__':
    app.run_server()

#### Create dash indicator update

In [10]:
def update_indicator(selected_uf):
    disease = 'dengue' # < pass this in the dropdown
    df = get_uf_by_disease(selected_uf, disease)
    traces = []

    # Select by uf
    df_uf = df['uf'] == selected_uf
    uf_selected = df[df_uf]

    # Count the total cities in uf
    cities_uf = len(uf_selected.nome.unique())
    print(cities_uf)

    # Filter nivel
    nivel_uf = 4
    rec_uf = uf_selected['nivel'] == nivel_uf
    rec_selected = uf_selected[rec_uf]

    # Filter by_week
    this_week = rec_selected.SE.max()
    last_week = this_week - 1
    df_this_week = rec_selected[rec_selected['SE'] == this_week].nivel.count()    
    df_last_week = rec_selected[rec_selected['SE'] == last_week].nivel.count()

    print(df_last_week)
#     print(df)

selected_uf = 'Ceará'
update_indicator(selected_uf)

184
1


#### Create Stack-bar

In [102]:
import plotly.offline as pyo
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px


In [229]:
def update_stack(selected_uf):
    disease = 'dengue' # < pass this in the dropdown
    df = get_uf_by_disease(selected_uf, disease)    
    
    # Select by uf
    df_uf = df['uf'] == selected_uf
    uf_selected = df[df_uf]
    keys=['nome']
    df_by_uf = uf_selected.groupby(['SE', 'nivel'])[keys].count()

    # Filter by_week
    this_week = df_by_uf.index.get_level_values(0).max()
    last_week = this_week - 1

    # Create df with filter by week
#     print(df_by_uf.loc[last_week])
#     print(df_by_uf.loc[this_week])

#     df_w = df_by_uf.loc[this_week]
    
    
    df_this_week = df_by_uf.loc[this_week].reset_index()
    print(df_this_week)
    

selected_uf = 'Paraná'
update_stack(selected_uf) 

   nivel  nome
0      1   392
1      2     3
2      3     3
3      4     1


In [268]:
def update_stack(selected_uf):

    df=pd.read_csv('dados_alerta.csv')

    # Select by uf
    df_uf = df['uf'] == selected_uf
    uf_selected = df[df_uf]
    keys = ['nome']
    df_by_uf = uf_selected.groupby(['SE', 'nivel'])[keys].count()

    # Filter by_week
    this_week = df_by_uf.index.get_level_values(0).max()
    last_week = this_week - 1

    # Create df_by_week
    df_this_week = df_by_uf.loc[this_week].reset_index()
    df_last_week = df_by_uf.loc[this_week].reset_index()

    fig = go.Figure()
    for nivel in df_this_week.nivel:
        
        fig.add_trace(go.Bar(
           x=df_this_week.nivel == nivel, 
           y=df_this_week.nome,
            name='This week',
        ))

    fig.update_layout(title_text='Least Used Feature', barmode='stack')

    pyo.plot(fig, filename='bar3.html')    



selected_uf = 'Paraná'
update_stack(selected_uf) 

In [194]:
import plotly.offline as pyo

fig = go.Figure()

ks_alert = [
    'alerta verde',
    'alerta amarelo',
    'alerta laranja',
    'alerta vermelho',
]
colors = [
    'rgb(0,255,0)',
    'rgb(255,255,0)',
    'rgb(255,150,0)',
    'rgb(255,0,0)',
]
    
niveles = df_last_se.nivel.unique()
for nivel, c, ks in zip(niveles, colors, ks_alert ):
            fig.add_trace(
                    go.Bar(
                        x=uf_selected.SE,  # NOC stands for National Olympic Committee
                        y=uf_selected.nome,
                        name = ks,
                        marker={'color': c},
    ))

            fig.update_layout(title_text='Least Used Feature', barmode='stack')

pyo.plot(fig, filename='bar3.html')

NameError: name 'df_last_se' is not defined

In [233]:
   nivel  nome
0      1   392
1      2     3
2      3     3
3      4     1

SyntaxError: invalid syntax (<ipython-input-233-440cf4bf5529>, line 1)